In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost
!pip install hyperopt

In [0]:
import pandas  as pd
import numpy  as  np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import xgboost as xgb

import eli5
from eli5.sklearn import PermutationImportance

from hyperopt import hp, fmin, tpe, STATUS_OK

# Load Data

In [28]:

df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

# Code from day 4

In [0]:
# remove EUR
df = df[ df['price_currency'] != 'EUR' ]

# format features
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance(df[ feat ][0], list): continue

  factorized_values = df[ feat ].factorize()[0]
  # avoid columns named like 'feat___cat___cat'
  if SUFFIX_CAT in feat:
    df[ feat ] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]

# Convert strings to integers and none to -1
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm3')[0].replace(' ', '')))

In [33]:
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_faktura-vat__cat','param_moc',\
         'param_skrzynia-biegów__cat','feature_kamera-cofania__cat','param_marka-pojazdu__cat','param_typ__cat',\
         'param_pojemność-skokowa','feature_wspomaganie-kierownicy__cat','seller_name__cat','param_wersja__cat',\
         'param_model-pojazdu__cat','feature_światła-led__cat','feature_asystent-pasa-ruchu__cat','param_kod-silnika__cat',\
         'feature_system-start-stop__cat','feature_regulowane-zawieszenie__cat','feature_łopatki-zmiany-biegów__cat']
# function for model running
def run_model(model, feats):
  X  =  df[  feats  ].values
  y = df[ 'price_value' ].values

  scores = cross_val_score(model, X, y, cv = 2, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)


xgb_params = {
    'max_depth' : 5,
    'n_estimators' : 50,
    'learning_rate': 0.1,
    'seed':0
}
xgb_model = xgb.XGBRegressor(**xgb_params) 

# run model on modified features
run_model(xgb_model, feats)

[21:28:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:28:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9550.300371001322, 12.060715279800206)

# Hyperparameters optimization

In [36]:
def obj_func(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBRegressor(**params), feats)
  return {'loss': np.abs(mean_mae), 'status': STATUS_OK}

xgb_reg_params = {
      'learning_rate': hp.choice('learning_rate', np.arange(0.05, 0.31, 0.05)),
      'max_depth':     hp.choice('max_depth', np.arange(5, 16, 1, dtype=int)),
      'subsample':     hp.quniform('subsample', 0.5, 1, 0.05),
      'colsample_bytree':     hp.quniform('colsample_bytree', 0.5, 1, 0.05),
      'objective':     'reg:squarederror',
      'n_estimators':  100,
      'seed':          0,
}
best = fmin(obj_func, xgb_reg_params, algo=tpe.suggest, max_evals=10)
best

Training with params: 
{'colsample_bytree': 0.8, 'learning_rate': 0.25, 'max_depth': 7, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.7000000000000001}
Training with params: 
{'colsample_bytree': 0.9, 'learning_rate': 0.3, 'max_depth': 13, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Training with params: 
{'colsample_bytree': 0.8500000000000001, 'learning_rate': 0.3, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.9}
Training with params: 
{'colsample_bytree': 0.65, 'learning_rate': 0.2, 'max_depth': 12, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65}
Training with params: 
{'colsample_bytree': 1.0, 'learning_rate': 0.25, 'max_depth': 9, 'n_estimators': 100, 'objective': 'reg:squarederror', 'seed': 0, 'subsample': 0.65}
Training with params: 
{'colsample_bytree': 1.0, 'learning_rate': 0.15000000000000002, 'max_depth': 12, 'n_estim

{'colsample_bytree': 1.0,
 'learning_rate': 2,
 'max_depth': 7,
 'subsample': 0.9500000000000001}

In [0]:
!git add day